In [1]:
import numpy as np
import torch.nn.functional as F
import torch
import torch.nn as nn
from torchsummary import summary

In [2]:
# for now just generate some random data, don't have access to kaggle at work
CLASSES = ['fine_concrete', 'concrete', 'soft_tiles', 'tiled', 'soft_pvc',
           'hard_tiles_large_space', 'carpet', 'hard_tiles', 'wood']
y_train = np.random.choice(CLASSES, size=(3810,), replace=True)
x_train = np.random.rand(3810,10,128)

In [79]:
class ShallowCNN(nn.Module):
    def __init__(self):
        super(ShallowCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=40, kernel_size=(1,10))
        self.conv2 = nn.Conv2d(in_channels=40, out_channels=40, kernel_size=(10,1))
        self.conv2_bn = nn.BatchNorm2d(40)
        self.max_pool = nn.MaxPool2d(kernel_size=(1,25), stride=(1,5))
        self.linear = nn.Linear(in_features=40*19, out_features=10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.elu(self.conv2_bn(x))
        x = x.view((-1,40,119))
        x = self.max_pool(x)
        x = x.view((x.size(0), -1))
        x = self.linear(x)
        return x

In [ ]:
'''
class ShallowCNN(nn.Module):
    def __init__(self, n_conv_channels=40, 
                 n_signal_channels=10, 
                 pool_size=25, 
                 pool_stride=15,
                 n_out=10):
        
        super(ShallowCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=n_conv_channels, 
                               kernel_size=(1,n_signal_channels))
        
        self.conv2 = nn.Conv2d(in_channels=n_conv_channels, 
                               out_channels=n_conv_channels, 
                               kernel_size=(n_signal_channels,1))
        
        self.conv2_bn = nn.BatchNorm2d(n_signal_channels)
        
        self.max_pool = nn.MaxPool2d(kernel_size=(1,pool_size), stride=(1,pool_stride))
        # Need to determine how to calculate 7
        self.linear = nn.Linear(in_features=40*7, out_features=n_out)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.elu(self.conv2_bn(x))
        # need to determine how to calculate this size
        x = x.view((-1,40,119))
        x = self.max_pool(x)
        x = x.view((x.size(0), -1))
        x = self.linear(x)
        return x
'''

In [80]:
foo = ShallowCNN()
foo.cuda()

ShallowCNN(
  (conv1): Conv2d(1, 40, kernel_size=(1, 10), stride=(1, 1))
  (conv2): Conv2d(40, 40, kernel_size=(10, 1), stride=(1, 1))
  (conv2_bn): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool): MaxPool2d(kernel_size=(1, 25), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=760, out_features=10, bias=True)
)

In [81]:
summary(foo, input_size=x_train[0:1].shape)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 40, 10, 119]             440
            Conv2d-2           [-1, 40, 1, 119]          16,040
       BatchNorm2d-3           [-1, 40, 1, 119]              80
         MaxPool2d-4               [-1, 40, 19]               0
            Linear-5                   [-1, 10]           7,610
Total params: 24,170
Trainable params: 24,170
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.44
Params size (MB): 0.09
Estimated Total Size (MB): 0.54
----------------------------------------------------------------


In [ ]:
def get_data_loader(batchsize, file_dir)

In [82]:
import torchvision.datasets as datasets

In [83]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

Processing...
Done!


In [86]:
mnist_trainset.train_labels.shape

torch.Size([60000])

In [92]:
import pandas as pd
import torch
import numpy as np 
from os.path import join
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder



X_TRAIN = 'X_train.csv'
X_TEST = 'X_test.csv'
Y_TRAIN = 'y_train.csv'

FEATURE_COLS = ['orientation_X',
                'orientation_Y', 'orientation_Z', 'orientation_W', 'angular_velocity_X',
                'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X',
                'linear_acceleration_Y', 'linear_acceleration_Z']


def x_to_array(df, feature_cols=FEATURE_COLS):
    # Convert tabular data into 3D numpy array
    n_series = len(df.series_id.unique())
    n_features = len(feature_cols)
    n_samples = max(df.measurement_number+1)
    x_trans = np.empty((n_series, n_samples, n_features))
    for i, idee in enumerate(df.series_id.unique()):
        x_trans[i] = np.array(df[df.series_id == idee][feature_cols])
    return x_trans
    

def load_and_format_data(path):
    x_train = pd.read_csv(join(path,X_TRAIN))
    x_test = pd.read_csv(join(path,X_TEST))
    y_train = pd.read_csv(join(path,Y_TRAIN))

    x_test = x_to_array(x_test)
    x_train = x_to_array(x_train)
    y_train = y_train.surface # for now just return unencoded values, let Tensorflow take care of this

    return x_train, y_train, x_test


class RobotNavDataset(Dataset):
    # TODO add transformations here
    # TODO add option to one hot labels
    def __init__(self, path_dir):
        self.train, self.labels, self.test = self.load_and_format_data(path_dir)
        
    def load_and_format_data(self, path):
        x_train = pd.read_csv(join(path,X_TRAIN))
        x_test = pd.read_csv(join(path,X_TEST))
        y_train = pd.read_csv(join(path,Y_TRAIN))

        x_test = x_to_array(x_test)
        x_train = x_to_array(x_train)
        y_train = y_train.surface # for now just return unencoded values, let Tensorflow take care of this
        y_train = LabelEncoder().fit_transform(y_train)

        return torch.tensor(x_train), torch.tensor(y_train), torch.tensor(x_test)
        

In [94]:
data = RobotNavDataset('../../../data/')

FileNotFoundError: File b'../../../data/X_train.csv' does not exist